In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import precision_recall_curve
def load_data():
    return pd.read_csv("dataset/D48.csv",index_col=0)

In [2]:
data_set=load_data()

In [3]:
import hashlib
def test_set_check(identifier,test_ratio,hash):
    return bytearray(hash(np.int64(identifier)).digest())[-1]  < 51
def split_train_test_by_id(data,test_ratio,id_column,hash):
    ids=data[id_column]
    in_test_set=ids.apply(lambda id_:test_set_check(id_,test_ratio,hash))
    return in_test_set

In [4]:
data_set=data_set.reset_index()
in_test_set=split_train_test_by_id(data_set,0.66,"index",hashlib.md5)
test_set=data_set[in_test_set]
train_set=data_set[~in_test_set]



In [5]:
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy="median")
imputer.fit(train_set)
train_set=imputer.transform(train_set)
train_set_array=train_set

#train_set_array=train_set.as_matrix()
test_set_copy=test_set.copy()
del test_set['9']
test_set_array=test_set.as_matrix()


In [6]:
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[9] not in separated):
            separated[vector[9]] = []
        separated[vector[9]].append(vector)
    return separated



In [7]:
import math
import sys
def mean(numbers):
    return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

def summarize(dataset):
    zipd=zip(*dataset)
    del zipd[9]
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zipd]
    return summaries


def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.iteritems():
        summaries[classValue] = summarize(instances)
    return summaries

summary = summarizeByClass(train_set_array)


In [8]:
import math
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.iteritems():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.iteritems():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions



In [9]:
predictions=getPredictions(summary,test_set_array)


In [10]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][9] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [11]:
test_set_copy=test_set_copy.as_matrix()
accuracy=getAccuracy(test_set_copy,predictions)
print(accuracy)


82.619047619
